In [2]:
import warnings
warnings.filterwarnings('ignore')
import keras
import pandas as pd
from sklearn.model_selection import train_test_split
import re, string, unicodedata
import nltk
#nltk.download('popular')
from nltk.stem.snowball import SnowballStemmer
#import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import numpy as np
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Flatten,Input 
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.optimizers import SGD
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
from numpy import mean, std
from sklearn.metrics import classification_report
import plotly.graph_objs as go
import plotly.plotly as py
#import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
import matplotlib.pyplot as plt
from sklearn.utils import class_weight

In [66]:
data = pd.read_csv('google-maps-single-dataset-for-conference.csv',encoding='ISO-8859-1')
data= data[pd.notnull(data['Claim_Type'])]
print(len(data))

data = data.replace("neu ", "neu")
data = data.replace("supporting ", "supporting")
data = data.replace("attacking ", "attacking")
data = data.replace("atacking ", "attacking")
data = data.replace("attacknig ", "attacking")

data= data[data.Rationale_Type=='claim']
print(data.Claim_Type.value_counts())
#data = data.replace("issue ", "issue")
data.head()

2077
neu           772
supporting    762
attacking     542
Name: Claim_Type, dtype: int64


,comment_Text,comment_author,comment_id,comment_upvotes,parent_id,Rationale_Type,Claim_Type,Have_Rationale
1,People still do racial checkbox ads?,PubicLouseInDaHouse,dfadazi,-1,60v5za,claim,neu,no
2,What?,Isvara,dfade5q,1,dfadazi,claim,neu,no
3,90s power rangers.,PubicLouseInDaHouse,dfadlgi,1,dfade5q,claim,neu,no
5,https://support.google.com/plus/answer/2998354...,sparr,dfa365t,-1,60v5za,claim,supporting,yes
6,This is really cool for people who have friends.,Unidan_nadinU,df9vhad,-1,60v5za,claim,supporting,yes


In [67]:
def print_plot(index):
    example = data[data.index == index][['comment_Text', 'Rationale_Type']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Rationale_Type:', example[1])


In [68]:
print_plot(60)

> gender equality in any and all professions.

Hahah uh... what?

EDIT: Also I wasn't the one to downvote you above. No clue what that's about.
Rationale_Type: claim


In [69]:
#df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\<\>\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z =#+_]')
#STOPWORDS = set(stopwords.words('english'))

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_url(text):
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    return text

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = strip_html(text) 
    text = remove_url(text)
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    #text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    #text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
data['comment_Text'] = data['comment_Text'].apply(clean_text)
#df['Consumer complaint narrative'] = df['Consumer complaint narrative'].str.replace('\d+', '')

In [70]:
print_plot(60)

  gender equality in any and all professionshahah uh whatedit also i wasnt the one to downvote you above no clue what thats about
Rationale_Type: claim


In [71]:
print(data.Claim_Type.value_counts())
#X= data.comment_Text.values.astype('U')
X= data.comment_Text.values.astype('U')
y=data.Claim_Type.values.astype('U')
rationale_type=data.Claim_Type
rationale_type_name= data.Claim_Type.unique()
print(rationale_type_name)

neu           772
supporting    762
attacking     542
Name: Claim_Type, dtype: int64
['neu' 'supporting' 'attacking']


In [72]:
macronum=sorted(set(data['Claim_Type']))
print(macronum)
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

data['Claim_Type']=data['Claim_Type'].apply(fun)
#print(data['Rationale_Type'])

['attacking', 'neu', 'supporting']


In [73]:
rationale_labels = []
for idx in data['Claim_Type']:
    rationale_labels.append(idx)

In [74]:
vocab_size= 10000
MAX_SEQUENCE_LENGTH = 1000
tokenizer= Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X)
sequences_ANN = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index
print('Number of Unique Tokens',len(word_index))
final_data_ANN = pad_sequences(sequences_ANN, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
#print(final_data_ANN)
rationale_labels_ANN=to_categorical(np.asarray(rationale_labels))
print(final_data_ANN.shape, rationale_labels_ANN.shape)

Number of Unique Tokens 5883
(2076, 1000) (2076, 3)


In [75]:
vocab_size= 10000
MAX_SEQUENCE_LENGTH = 1000
batch_size= 12
epochs= 10

test_accuracy =[]
report_accuracy= []

def evaluate_ANN_Model(trainX,trainy, testX, testy):
    
    
    embeddings_index = {}
    f = open('glove.6B.100d.txt',encoding='utf8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))
    
    
    EMBEDDING_DIM=100
    MAX_SEQUENCE_LENGTH = 1000
    #embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    embedding_matrix = zeros((len(word_index) + 1, EMBEDDING_DIM))

    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
            
    embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
    
    
    smote = SMOTE('minority')
    X_sm_embedding, y_sm_embedding = smote.fit_sample(trainX, trainy)
    
    
    
    model = Sequential()
    
    model.add(embedding_layer) 
    model.add(Flatten())
    #input layer
    model.add(Dense(16))
    model.add(Activation('relu'))
    model.add(Dropout(0.35))
    
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer= 'SGD', metrics=['accuracy'])  
    
    #class_weight= class_weight.compute_class_weight('balanced', np.unique(rationale_type),rationale_type)
    
   
    #model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])
     
    model.fit(X_sm_embedding,y_sm_embedding, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2)
    

    #evaluate model
    score= model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
    test_accuracy= score[1]
    #predict model accuracy on testing data
    pred = model.predict(testX, batch_size=12, verbose=1)
    predicted = np.argmax(pred, axis=1)
    report_accuracy = precision_recall_fscore_support(np.argmax(testy, axis=1), predicted)
    return model, test_accuracy, report_accuracy
    

In [76]:
#Kfold ANN with TFIDF

kf= KFold(n_splits=3)
#curr_fold= 0
alg_accurcy_ANN_embedding =[]
alg_testing_accurcy_ANN_embedding =[]

for train_idx, test_idx in kf.split(final_data_ANN):
    X_train, X_test= final_data_ANN[train_idx], final_data_ANN[test_idx]
    y_train, y_test= rationale_labels_ANN[train_idx], rationale_labels_ANN[test_idx]
        
    model, test_accuracy,report_accuracy = evaluate_ANN_Model(X_train,y_train, X_test, y_test )
    print ("the training accuracy is",test_accuracy )
    alg_accurcy_ANN_embedding.append(test_accuracy)
    print ("the testing accuracy is",report_accuracy )
    alg_testing_accurcy_ANN_embedding.append(report_accuracy)
    

average_accuracy_ANN_training= np.mean(alg_accurcy_ANN_embedding, axis=0)
print("the average Training Accuracy of ANN-embedding is",average_accuracy_ANN_training) 

average_accuracy_ANN_testing= np.mean(alg_testing_accurcy_ANN_embedding, axis=0)
print("the averag Validation accuracy of ANN-embedding is",average_accuracy_ANN_testing)



Total 400000 word vectors in Glove 6B 100d.
Train on 1270 samples, validate on 318 samples
Epoch 1/10
1270/1270 [==============================] - 8s 6ms/step - loss: 1.0944 - acc: 0.3780 - val_loss: 1.2011 - val_acc: 0.1824
Epoch 2/10
1270/1270 [==============================] - 3s 2ms/step - loss: 1.0616 - acc: 0.4197 - val_loss: 1.2004 - val_acc: 0.1918
Epoch 3/10
1270/1270 [==============================] - 2s 2ms/step - loss: 1.0389 - acc: 0.4504 - val_loss: 1.0981 - val_acc: 0.3836
Epoch 4/10
1270/1270 [==============================] - 2s 2ms/step - loss: 1.0030 - acc: 0.4992 - val_loss: 1.1238 - val_acc: 0.28620s - loss: 1.0079 - acc: 0.5 - ETA: 0s - loss: 1.0045
Epoch 5/10
1270/1270 [==============================] - 2s 2ms/step - loss: 0.9553 - acc: 0.5559 - val_loss: 1.3349 - val_acc: 0.1918
Epoch 6/10
1270/1270 [==============================] - 2s 2ms/step - loss: 0.9146 - acc: 0.5756 - val_loss: 1.1367 - val_acc: 0.3459
Epoch 7/10
1270/1270 [==============================